<a href="https://colab.research.google.com/github/Carlos-dev-G/Redes-convolucional-Simples/blob/main/CNN_data_set_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import kagglehub

# Descargar el conjunto de datos de tipos de cabello
path = kagglehub.dataset_download("kavyasreeb/hair-type-dataset")
print("Ruta a los archivos del conjunto de datos:", path)

# Definir la ruta a los datos (ajusta según la estructura del conjunto de datos)
data_dir = os.path.join(path, "data")

# Configurar los generadores de datos
# Aumento de datos para el conjunto de entrenamiento
datagen_entrenamiento = ImageDataGenerator(
    rescale=1./255,                # Normalizar las imágenes
    rotation_range=20,             # Rotar imágenes aleatoriamente
    width_shift_range=0.2,         # Desplazar horizontalmente
    height_shift_range=0.2,        # Desplazar verticalmente
    shear_range=0.2,               # Aplicar corte
    zoom_range=0.2,                # Zoom aleatorio
    horizontal_flip=True,          # Invertir horizontalmente
    fill_mode='nearest'            # Rellenar con el valor más cercano
)

# Generador para el conjunto de validación
datagen_validacion = ImageDataGenerator(rescale=1./255)  # Solo normalización

# Crear generadores de datos
datos_entrenamiento = datagen_entrenamiento.flow_from_directory(
    data_dir,
    target_size=(224, 224),        # Redimensionar las imágenes
    batch_size=32,
    class_mode='sparse',           # Para clasificación multiclase
    shuffle=True
)

datos_validacion = datagen_validacion.flow_from_directory(
    data_dir,
    target_size=(224, 224),        # Redimensionar las imágenes
    batch_size=32,
    class_mode='sparse',           # Para clasificación multiclase
    shuffle=False
)

# Obtener el número de clases y los nombres de las clases
nombres_clases = datos_entrenamiento.class_indices
num_clases = len(nombres_clases)

# Crear el modelo de red neuronal
modelo = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),  # Tamaño de imagen reducido
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Conv2D(256, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(512, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(num_clases, activation='softmax')
])

# Compilar el modelo
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Entrenar el modelo
historial = modelo.fit(
    datos_entrenamiento,
    epochs=15,
    validation_data=datos_validacion
)

In [ ]:
# Visualizar algunas predicciones
def mostrar_predicciones(generator, num=9):
    imagenes, etiquetas = next(generator)
    predicciones = modelo.predict(imagenes)

    plt.figure(figsize=(10, 10))
    for i in range(num):
        plt.subplot(3, 3, i + 1)
        plt.imshow(imagenes[i])  # Mostrar la imagen
        plt.title(f"Predicción: {list(nombres_clases.keys())[tf.argmax(predicciones[i]).numpy()]} \n Real: {list(nombres_clases.keys())[int(etiquetas[i])]}")
        plt.axis('off')
    plt.show()

# Visualizar algunas predicciones del conjunto de validación
mostrar_predicciones(datos_validacion)